In [16]:

import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import tensorboard
from datetime import datetime
from packaging import version
from tensorflow import keras




In [17]:
mobile = keras.applications.mobilenet.MobileNet()
print(mobile.summary())
print("model loaded")

17227776/17225924 [==============================] - 3s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________

In [49]:
x = mobile.get_layer('global_average_pooling2d')
output = Dense(units=1, activation='sigmoid')(x.output)
model = Model(inputs=mobile.input, outputs=output)
for i in model.layers[:-5]:
    i.trainable = False


In [64]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
 
train_generator=train_datagen.flow_from_directory('./DATASET/TRAIN', #path of dataset folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True)


test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
                                                        './DATASET/TEST',
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [65]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [67]:

step_size_train=train_generator.n//train_generator.batch_size
model.fit(train_generator,
                   steps_per_epoch=100,
                   epochs=5)

Epoch 1/5
100/100 [==============================] - 75s 753ms/step - loss: 0.1464 - accuracy: 0.9494
Epoch 2/5
100/100 [==============================] - 72s 716ms/step - loss: 0.1482 - accuracy: 0.9472
Epoch 3/5
100/100 [==============================] - 71s 711ms/step - loss: 0.1301 - accuracy: 0.9519
Epoch 4/5
100/100 [==============================] - 73s 732ms/step - loss: 0.1242 - accuracy: 0.9547
Epoch 5/5
100/100 [==============================] - 63s 628ms/step - loss: 0.1130 - accuracy: 0.9628


In [68]:
model.save_weights("model_weights.h5")
model.save('model.h5')

In [72]:
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model ./model/model.h5  ./json-model    

2021-05-20 12:57:18.232610: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-05-20 12:57:18.232636: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
